In [ ]:
#MA50 PullOver

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import datetime
import yfinance as yf
import math
import talib as ta


In [2]:
def smaGoingUp(sma):
    return sma.is_monotonic

In [3]:
def backtest50MAPull(df, ticker, investment=5000):
    close = df['Close']
    high = df['High']
    low = df['Low']
    open = df['Open']
    sma50 = df['SMA50']
    date = df['Date']

    # initial condition
    #investment = 5000
    in_position = False
    equity = investment
    commission = 50
    share = 0
    entry = []
    exit = []
    tradeCount=0
    buyDate = ''
    sellDate = ''
    buyPrice = 0
    sellPrice = 0

    i = 11
    while i < (len(df)-1):
        # if not in position & price is on uptrend -> buy
        if not in_position and (close[i-1] < sma50[i-1]) and low[i-1] == low[i-11:i].min() and high[i]> high[i-1] and low[i]> low[i-1]:
            stopLimit = high[i]
            lowerLimit = low[i-1]
            rangeLast = min(4,len(df)-i)
            diff = stopLimit - lowerLimit
            sl = high[i] * 0.97
            stopLoss = max(low[i-1],sl)
            #stopLoss = low[i-1]
            target = high[i] + (diff * 2)
            for j in range(1,rangeLast):
                i=i+1;
                if high[i] > stopLimit:
                    share = math.floor(equity / stopLimit )
                    equity -= share * stopLimit
                    buyDate = date[i]
                    entry.append((i, stopLimit, buyDate))
                    in_position = True
                    tradeCount += 1
                    #print(f'Buy {share} shares at {round(high[i+j-1],2)} on {df.index[i].strftime("%Y/%m/%d")}')
                    break;


        elif in_position and ((close[i] > target) or (close[i] < stopLoss)):
            equity += share * open[i+1] - commission
            sellDate = date[i+1]
            exit.append((i+1, open[i+1], sellDate, equity))
            in_position = False
            #print(f'Sell at {round(open[i+1],2)} on {df.index[i+1].strftime("%Y/%m/%d")}')
        

        i = i+1


    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'{ticker} :\t Earning from investing {investment} is ${round(earning,2)} \t (ROI = {roi}%) \t Trades={tradeCount}')
    return entry, exit, earning, roi, tradeCount

In [4]:
def backtest50MAPullWithTrail(df, ticker, investment=5000):
    close = df['Close']
    high = df['High']
    low = df['Low']
    open = df['Open']
    sma50 = df['SMA50']
    #sma9 = df['SMA9']
    date = df['Date']

    # initial condition
    # investment = 5000
    in_position = False
    equity = investment
    commission = 50
    share = 0
    entry = []
    exit = []
    tradeCount = 0
    buyDate = ''
    sellDate = ''
    buyPrice = 0
    sellPrice = 0
    multiplier = 3;
    targetHit = False;

    i = 11
    while i < (len(df) - 1):
        # if not in position & price is on uptrend -> buy
        currentDate = date[i]
        if not in_position and (close[i - 1] < sma50[i - 1]) and low[i - 1] == low[i - 11:i].min() and high[i] > high[
            i - 1] and low[i] > low[i - 1]:
            multiplier = 2;
            targetHit = False;
            stopLimit = high[i]
            lowerLimit = low[i - 1]
            rangeLast = min(4, len(df) - i)
            diff = stopLimit - lowerLimit
            sl = high[i] * 0.97
            stopLoss = max(low[i - 1], sl)
            target = high[i] + (diff * multiplier)
            signalDate = date[i]
            trailSL = target
            t1 = target
            t2 = high[i] + (diff * 3)
            t3 = high[i] + (diff * 4)
            t4 = high[i] + (diff * 5)
            for j in range(1, rangeLast):
                i = i + 1;
                if high[i] > stopLimit:
                    share = math.floor(equity / stopLimit)
                    equity -= share * stopLimit
                    buyDate = date[i]
                    entry.append((i, stopLimit, buyDate))
                    in_position = True
                    tradeCount += 1
                    # print(f'Buy {share} shares at {round(high[i+j-1],2)} on {df.index[i].strftime("%Y/%m/%d")}')
                    break;

        elif in_position and ((targetHit and close[i] < trailSL) or (close[i] < stopLoss)):
            equity += share * open[i + 1] - commission
            sellDate = date[i + 1]
            exit.append((i + 1, open[i + 1], sellDate, equity))
            in_position = False
            # print(f'Sell at {round(open[i+1],2)} on {df.index[i+1].strftime("%Y/%m/%d")}')

        if in_position and close[i] > target:
            targetDate = date[i]
            targetHit = True;
            trailSL = stopLimit + (diff * (multiplier-1))
            multiplier = multiplier + 1
            target = stopLimit + (diff * multiplier)

        i = i + 1

    earning = equity - investment
    roi = round(earning / investment * 100, 2)
    print(
        f'{ticker} :\t Earning from investing {investment} is ${round(earning, 2)} \t (ROI = {roi}%) \t Trades={tradeCount}')
    return entry, exit, earning, roi, tradeCount


In [ ]:
startDate='2014-11-01'
ticker = "SBIN" + ".NS" 
df = yf.download(ticker, start=startDate) 
df['Date'] = df.index
df['SMA50'] = ta.SMA(df['Close'], timeperiod=50)
df['SMA9'] = ta.SMA(df['Close'], timeperiod=9)
df.dropna(inplace=True)

ROI =[]
tickerList =[]
investment = []
totalTrades = []
finalResults = []
Earning = []
entry, exit, earning, roi, tradeCount = backtest50MAPullWithTrail(df, ticker, 10000)
Earning.append(earning)
ROI.append(roi)
investment.append(10000)
totalTrades.append(tradeCount)
tickerList.append(ticker)
entryDF = pd.DataFrame(entry,columns=['Day','BuyPrice','BuyDate'])
exitDF = pd.DataFrame(exit,columns=['Day','SellPrice','SellDate','Equity'])
result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
result['Symbol'] = ticker = "SBIN"
result.drop(['Day_x', 'Day_y'], axis = 1,inplace=True)
result.to_csv('result\MA50_PullOver_Result.csv')

In [5]:
# Fetch Daily data for give tickerList

tickerList=['IOB', 'NHPC', 'UNIONBANK', 'IDBI', 'PNB', 'IDFCFIRSTB', 'BHEL', 'BANKINDIA', 'BANKBARODA', 'L&TFH', 'ADANIPOWER', 'TATAPOWER', 'NTPC', 'IOC', 'ONGC', 'SAIL', 'ABCAPITAL', 'ASHOKLEY', 'COALINDIA', 'CANBK', 'TATAMTRDVR', 'GAIL', 'NMDC', 'MOTHERSUMI', 'VEDL', 'HINDPETRO', 'TATAMOTORS', 'AMBUJACEM', 'HINDZINC', 'HINDALCO', 'JINDALSTEL', 'CROMPTON', 'SBIN', 'TORNTPOWER', 'LICHSGFIN', 'MARICO', 'BPCL', 'GUJGASLTD', 'IGL', 'LAURUSLABS', 'BHARTIARTL', 'DABUR', 'ICICIPRULI', 'WIPRO', 'CHOLAFIN', 'SUNTV', 'MCDOWELL-N', 'CADILAHC', 'TATACONSUM', 'CONCOR', 'ICICIBANK', 'TVSMOTOR', 'HDFCLIFE', 'BHARATFORG', 'JSWSTEEL', 'SUNPHARMA', 'AXISBANK', 'ADANIPORTS', 'CUMMINSIND', 'BERGEPAINT', 'UPL', 'M&M', 'TRENT', 'GODREJCP', 'CIPLA', 'RAMCOCEM', 'HCLTECH', 'SBILIFE', 'AUBANK', 'INDUSINDBK', 'VOLTAS', 'HAVELLS', 'AUROPHARMA', 'SBICARD', 'TECHM', 'TATASTEEL', 'LUPIN', 'MUTHOOTFIN', 'ADANIGREEN', 'ADANIENT', 'GODREJPROP', 'ATGL', 'INFY', 'GRASIM', 'ADANITRANS', 'SRTRANSFIN', 'HDFCBANK', 'LT', 'TITAN', 'AARTIIND', 'PEL', 'KOTAKBANK', 'DEEPAKNTR', 'IRCTC', 'ACC', 'PIDILITIND', 'HINDUNILVR', 'MINDTREE', 'HDFC', 'EICHERMOT', 'ASIANPAINT']
nseTickerList=[]

tempList = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1].Symbol.to_list()
tickerList = tickerList + tempList
tickerList = list(set(tickerList))
startDate='2015-01-01'

for i in tickerList:
    nseTickerList.append(i+'.NS')

nseTickerList.remove('HDFCLIFE.NS')
nseTickerList.remove('SBICARD.NS')

nseDF=[] 
for ticker in nseTickerList: 
    data = []
    df = yf.download(ticker, start=startDate) 
    df['Date'] = df.index
    df['SMA50'] = ta.SMA(df['Close'], timeperiod=50)
    df['SMA9'] = ta.SMA(df['Close'], timeperiod=9)
    df.dropna(inplace=True)
    data.append(ticker)
    data.append(df)
    nseDF.append(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
# Fetch Intraday data for give tickerList

tickerList=['IOB', 'NHPC', 'UNIONBANK', 'IDBI', 'PNB', 'IDFCFIRSTB', 'BHEL', 'BANKINDIA', 'BANKBARODA', 'L&TFH', 'ADANIPOWER', 'TATAPOWER', 'NTPC', 'IOC', 'ONGC', 'SAIL', 'ABCAPITAL', 'ASHOKLEY', 'COALINDIA', 'CANBK', 'TATAMTRDVR', 'GAIL', 'NMDC', 'MOTHERSUMI', 'VEDL', 'HINDPETRO', 'TATAMOTORS', 'AMBUJACEM', 'HINDZINC', 'HINDALCO', 'JINDALSTEL', 'CROMPTON', 'SBIN', 'TORNTPOWER', 'LICHSGFIN', 'MARICO', 'BPCL', 'GUJGASLTD', 'IGL', 'LAURUSLABS', 'BHARTIARTL', 'DABUR', 'ICICIPRULI', 'WIPRO', 'CHOLAFIN', 'SUNTV', 'MCDOWELL-N', 'CADILAHC', 'TATACONSUM', 'CONCOR', 'ICICIBANK', 'TVSMOTOR', 'HDFCLIFE', 'BHARATFORG', 'JSWSTEEL', 'SUNPHARMA', 'AXISBANK', 'ADANIPORTS', 'CUMMINSIND', 'BERGEPAINT', 'UPL', 'M&M', 'TRENT', 'GODREJCP', 'CIPLA', 'RAMCOCEM', 'HCLTECH', 'SBILIFE', 'AUBANK', 'INDUSINDBK', 'VOLTAS', 'HAVELLS', 'AUROPHARMA', 'SBICARD', 'TECHM', 'TATASTEEL', 'LUPIN', 'MUTHOOTFIN', 'ADANIGREEN', 'ADANIENT', 'GODREJPROP', 'ATGL', 'INFY', 'GRASIM', 'ADANITRANS', 'SRTRANSFIN', 'HDFCBANK', 'LT', 'TITAN', 'AARTIIND', 'PEL', 'KOTAKBANK', 'DEEPAKNTR', 'IRCTC', 'ACC', 'PIDILITIND', 'HINDUNILVR', 'MINDTREE', 'HDFC', 'EICHERMOT', 'ASIANPAINT']
nseTickerList=[]

tempList = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1].Symbol.to_list()
tickerList = tickerList + tempList
tickerList = list(set(tickerList))

d = dt.today() - datetime.timedelta(days=50)
startDate = d.date().strftime("%Y-%m-%d")

for i in tickerList:
    nseTickerList.append(i+'.NS')

nseTickerList.remove('HDFCLIFE.NS')
nseTickerList.remove('SBICARD.NS')

nseDF=[] 
for ticker in nseTickerList: 
    data = []
    df = yf.download(ticker, start=startDate,interval='30m') 
    df['Date'] = df.index
    df['SMA50'] = ta.SMA(df['Close'], timeperiod=50)
    df['SMA9'] = ta.SMA(df['Close'], timeperiod=9)
    df.dropna(inplace=True)
    data.append(ticker)
    data.append(df)
    nseDF.append(data)

In [7]:
# Generate calls for NSE Data

ROI =[]
tickerList =[]
investment = []
totalTrades = []
finalResults = []
Earning = []
inv = 5000

for data in nseDF:
    df=data[1]
    ticker=data[0]
    entry, exit, earning, roi, tradeCount = backtest50MAPullWithTrail(df,ticker, inv)
    Earning.append(earning)
    ROI.append(roi)
    investment.append(inv)
    totalTrades.append(tradeCount)
    tickerList.append(ticker)
    entryDF = pd.DataFrame(entry,columns=['Day','BuyPrice','BuyDate'])
    exitDF = pd.DataFrame(exit,columns=['Day','SellPrice','SellDate','Equity'])
    result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
    result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
    result['Symbol'] = ticker
    result.drop(['Day_x', 'Day_y'], axis = 1,inplace=True)
    finalResults.append(result)
    
resultDF = pd.DataFrame(
    {'Ticker': tickerList,
     'Investment': investment,
     'FinalReturns': ROI,
     'TotalTrades': totalTrades
    })
resultDF['FinalAmount'] = resultDF['Investment'] + resultDF['Investment']*(resultDF['FinalReturns']/100)

finalResults = pd.concat(finalResults)
finalResults
finalResults = finalResults[['Symbol','BuyDate', 'BuyPrice', 'SellDate', 'SellPrice','PnL_Percent','Equity']]
finalResults.to_csv('result\MA50_PullOver_BackTest_Trades.csv')
resultDF.to_csv('result\MA50_PullOver_Final_Result.csv')

BHARATFORG.NS :	 Earning from investing 5000 is $-1525.35 	 (ROI = -30.51%) 	 Trades=33
PEL.NS :	 Earning from investing 5000 is $-1587.56 	 (ROI = -31.75%) 	 Trades=23
GRASIM.NS :	 Earning from investing 5000 is $3180.42 	 (ROI = 63.61%) 	 Trades=29
RAMCOCEM.NS :	 Earning from investing 5000 is $7810.1 	 (ROI = 156.2%) 	 Trades=28
HINDALCO.NS :	 Earning from investing 5000 is $7338.0 	 (ROI = 146.76%) 	 Trades=32
ADANIGREEN.NS :	 Earning from investing 5000 is $27362.1 	 (ROI = 547.24%) 	 Trades=13
ATGL.NS :	 Earning from investing 5000 is $3340.85 	 (ROI = 66.82%) 	 Trades=6
RELIANCE.NS :	 Earning from investing 5000 is $22792.11 	 (ROI = 455.84%) 	 Trades=24
ABCAPITAL.NS :	 Earning from investing 5000 is $-2387.45 	 (ROI = -47.75%) 	 Trades=27
ULTRACEMCO.NS :	 Earning from investing 5000 is $-1511.2 	 (ROI = -30.22%) 	 Trades=27
BERGEPAINT.NS :	 Earning from investing 5000 is $6344.91 	 (ROI = 126.9%) 	 Trades=20
M&M.NS :	 Earning from investing 5000 is $-1275.9 	 (ROI = -25.52%) 	 

HINDZINC.NS :	 Earning from investing 5000 is $-1273.95 	 (ROI = -25.48%) 	 Trades=37
VOLTAS.NS :	 Earning from investing 5000 is $-3861.55 	 (ROI = -77.23%) 	 Trades=23
AARTIIND.NS :	 Earning from investing 5000 is $9925.43 	 (ROI = 198.51%) 	 Trades=18
DIVISLAB.NS :	 Earning from investing 5000 is $708.15 	 (ROI = 14.16%) 	 Trades=24
SRTRANSFIN.NS :	 Earning from investing 5000 is $-4390.87 	 (ROI = -87.82%) 	 Trades=32
SBIN.NS :	 Earning from investing 5000 is $10017.45 	 (ROI = 200.35%) 	 Trades=37
GUJGASLTD.NS :	 Earning from investing 5000 is $-4833.73 	 (ROI = -96.67%) 	 Trades=24
IDFCFIRSTB.NS :	 Earning from investing 5000 is $-4984.7 	 (ROI = -99.69%) 	 Trades=28
BPCL.NS :	 Earning from investing 5000 is $-4741.15 	 (ROI = -94.82%) 	 Trades=42
CADILAHC.NS :	 Earning from investing 5000 is $-3038.85 	 (ROI = -60.78%) 	 Trades=30
BAJFINANCE.NS :	 Earning from investing 5000 is $39564.35 	 (ROI = 791.29%) 	 Trades=20
IOB.NS :	 Earning from investing 5000 is $-4993.1 	 (ROI = -99

In [ ]:
#Lower TimeFrame



d = dt.today() - datetime.timedelta(days=50)
startDate = d.date().strftime("%Y-%m-%d")
ticker = "SBIN" + ".NS" 
df = yf.download(ticker, start=startDate,interval='90m') 
df['Date'] = df.index
df['SMA50'] = ta.SMA(df['Close'], timeperiod=50)
df['SMA9'] = ta.SMA(df['Close'], timeperiod=9)
df.dropna(inplace=True)

ROI =[]
tickerList =[]
investment = []
totalTrades = []
finalResults = []
Earning = []
entry, exit, earning, roi, tradeCount = backtest50MAPull(df, ticker, 10000)
Earning.append(earning)
ROI.append(roi)
investment.append(10000)
totalTrades.append(tradeCount)
tickerList.append(ticker)
entryDF = pd.DataFrame(entry,columns=['Day','BuyPrice','BuyDate'])
exitDF = pd.DataFrame(exit,columns=['Day','SellPrice','SellDate','Equity'])
result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
result['Symbol'] = ticker = "SBIN"
result.drop(['Day_x', 'Day_y'], axis = 1,inplace=True)
result.to_csv('result\MA50_PullOver_Result.csv')

In [ ]:
sr.is_monotonic